# Semantic Similarity

This notebook explores ways to compare two Georgian phrases. Given two short phrases, we want to calculate a metric that will measure the distance between them. This distance should be based on the likeliness of their meanings rather than the literal matching of their words.

Then we use these comparison methods for semantic search. Given a corpus and a query sentence, we want to find the most similar sentences.

In [1]:
!pip install transformers sentence-transformers fasttext python-Levenshtein

from sentence_transformers import SentenceTransformer, CrossEncoder, util
import Levenshtein as lev
from typing import List
import torch
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

# Sentence-Transformers

There are several multilingual models for semantic similarity available on [Sentence-Transformers](https://www.sbert.net/docs/pretrained_models.html). We used **paraphrase-multilingual-MiniLM-L12-v2**, a multilingual version of **paraphrase-MiniLM-L12-v2**, trained on parallel data for 50+ languages, and Georgian among them.

In [2]:
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

Loading a collection of Georgian proverbs as a sample text corpus and precalculating their embeddings with the above model.

In [3]:
corpus = json.load(open('./storage/texts.json'))
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [4]:
def find_similar(corpus: List[str], 
                 corpus_embedding: List[torch.Tensor], 
                 queries: List[str], 
                 embedder: SentenceTransformer, 
                 top_k: int):
  assert len(corpus) == len(corpus_embeddings)

  top_k = min(top_k, len(corpus))
  
  for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
      print(corpus[idx], "(Score: {:.4f})".format(score))


In [5]:
# Query sentences:
queries = [
    'ჯოჯოხეთი',
    'სიღარიბე',
    'სიკეთე', 
    'ბოროტება',
    ]

In [6]:
find_similar(corpus, corpus_embeddings, queries, embedder, 5)





Query: ჯოჯოხეთი

Top 5 most similar sentences in corpus:
ჯოჯოხეთს ერთი მაშხალა აკლდაო (Score: 0.8702)
ქრთამი ჯოჯოხეთს ანათებსო (Score: 0.8582)
ჯოჯოხეთში ძახილია და გაგონება კი - არა (Score: 0.8146)
ჯოჯოხეთს ერთი მუგუზალი აკლდა და ისიც დაუმატესო (Score: 0.7652)
ცდა ბედის მონახევრეა (Score: 0.7096)




Query: სიღარიბე

Top 5 most similar sentences in corpus:
სიზარმაცე სიღარიბის გასაღებიაო (Score: 0.9292)
ობოლი გაზრდას მოელის, ღარიბი - გამდიდრებასო (Score: 0.7076)
ღარიბის ლუკმა ტკბილია (Score: 0.6758)
ღარიბ კაცს ხეირიანი შვილი რომ ყავდეს, ღარიბი აღარ იქნებაო. (Score: 0.6472)
საოხრეზე ნაშოვნი სატიალოზე დაიხარჯაო (Score: 0.5648)




Query: სიკეთე

Top 5 most similar sentences in corpus:
ყველაფერი დაიქცევა კეთილი საქმის მეტი (Score: 0.7723)
მადლი ჰქენი, მარილიც მოაყარეო (Score: 0.7573)
საშოვარი კარგია, ნაშოვარი ძვირფასი (Score: 0.7453)
კარგ მთქმელსა კარგი გამგონიც უნდა (Score: 0.7125)
ღარიბის ლუკმა ტკბილია (Score: 0.7122)




Query: ბოროტება

Top 5 most similar sentences in corpus:
ჯოჯოხ

# Cross-Encoder

Instead of storing precomputed embeddings we can just concatenate sentences and let the model measure similarity at once. Such model is called [Cross Encoder](https://www.sbert.net/examples/training/cross-encoder/README.html?highlight=crossencoder)


![](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/Bi_vs_Cross-Encoder.png)


In [7]:
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
# model = CrossEncoder('cross-encoder/stsb-roberta-base')

In [8]:
def find_similar_ce(corpus: List[str],
                 queries: List[str], 
                 model: CrossEncoder, 
                 top_k: int):

  top_k = min(top_k, len(corpus))
  
  for query in queries:
    pairs = [(query, sent) for sent in corpus]
    scores = model.predict(pairs)
    top_results = torch.topk(torch.Tensor(scores), k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
      print(corpus[idx], "(Score: {:.4f})".format(score))


In [9]:
find_similar_ce(corpus, queries, model, 5)





Query: ჯოჯოხეთი

Top 5 most similar sentences in corpus:
ძაღლი ხსენებაზედაო (Score: 8.1331)
ძალა ერთობაშია (Score: 8.1331)
გამშველებელს მოხვდებაო (Score: 8.1331)
ქვეითი ცხენოსანს დასცინოდა (Score: 8.1283)
კაცი კაცის წამალიაო (Score: 8.1283)




Query: სიღარიბე

Top 5 most similar sentences in corpus:
ძაღლი ხსენებაზედაო (Score: 8.1331)
ძალა ერთობაშია (Score: 8.1331)
გამშველებელს მოხვდებაო (Score: 8.1331)
ქვეითი ცხენოსანს დასცინოდა (Score: 8.1283)
კაცი კაცის წამალიაო (Score: 8.1283)




Query: სიკეთე

Top 5 most similar sentences in corpus:
აგრე არ უნდა, თაყაო, შენ რომ მამული გაჰყაო. (Score: 8.3258)
თუ თვალით ვერ დაინახო, ყურით გაგონილს ნუ დასდევო (Score: 8.3238)
ვერიდებოდი ღოლოსა, შემომხვდა ღობის ბოლოსა. (Score: 8.3156)
სიკვდილმა სთქვა: ერთი ისეთი ვერ მოვკალი, რომ სხვა რამეს არ დააბრალონო (Score: 8.2829)
ვიდრე ჭკვიანი დაფიქრდეს, გიჟი ხიდს გადაირბენსო (Score: 8.2536)




Query: ბოროტება

Top 5 most similar sentences in corpus:
აგრე არ უნდა, თაყაო, შენ რომ მამული გაჰყაო. (Score: 8.389

# Levenstein Distance

Levenshtein distance is a metric for measuring the difference between two sequences. Distance between two words is the minimum number of single-character edits (insertions, deletions, or substitutions) required to get one word from the other.

We can use this metric to measure phrase similarity. That would let us save space for storing models and embeddings, and coputation time.

In [10]:
def lev_dist(str1: str, str2: str) -> float:
        if not str1: return 0
        if not str2: return 0
        
        max_len = max(len(str1), len(str2))
        distance_norm = lev.distance(str1, str2) / max_len

        return distance_norm

'''
  Given document string and query finds minimal 
  Levenstein Distance between fragments of document and query string.
'''
def min_lev_dist(doc, query):
  min_dist = float('inf')
  
  doc_words = doc.split(' ')
  query_words = query.split(' ')

  doc_word_count = len(doc_words)
  query_word_count = len(query_words)

  if doc_word_count <= query_word_count:
    return lev.distance(doc, query)

  for i in range(doc_word_count - query_word_count+1):
    doc_fragment = ' '.join(doc_words[i:i+query_word_count])
    if len(doc_fragment) == 0: continue

    cur_dist = lev_dist(doc_fragment, query)
    min_dist = min(min_dist, cur_dist)
  return min_dist


def find_similar_ld(corpus: List[str],
                    queries: List[str],
                    top_k: int):

  top_k = min(top_k, len(corpus))
  
  for query in queries:
    scores = [-min_lev_dist(sent, query) for sent in corpus]
    top_results = torch.topk(torch.Tensor(scores), k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
      print(corpus[idx], "(Score: {:.4f})".format(score))

In [11]:
find_similar_ld(corpus, queries, 5)





Query: ჯოჯოხეთი

Top 5 most similar sentences in corpus:
ცოდნა სამოთხეა და არცოდნა - ჯოჯოხეთი (Score: -0.0000)
ჯოჯოხეთში ძახილია და გაგონება კი - არა (Score: -0.1111)
ჯოჯოხეთს ერთი მაშხალა აკლდაო (Score: -0.1250)
ჯოჯოხეთს ერთი მუგუზალი აკლდა და ისიც დაუმატესო (Score: -0.1250)
ქრთამი ჯოჯოხეთს ანათებსო (Score: -0.1250)




Query: სიღარიბე

Top 5 most similar sentences in corpus:
სიზარმაცე სიღარიბის გასაღებიაო (Score: -0.2222)
მდიდარს უთხრეს: მშვიდობაშიო, ღარიბს უთხრეს – ვინ გაჩუქაო (Score: -0.3750)
ბავშვის პირით სიმართლე ღაღადებსო (Score: -0.3750)
სიხარბემ რა ნახა - ყველა დაჰკარგა (Score: -0.3750)
ობოლი გაზრდას მოელის, ღარიბი - გამდიდრებასო (Score: -0.3750)




Query: სიკეთე

Top 5 most similar sentences in corpus:
გაცემულ სიკეთეს ხალხი არ ივიწყებს (Score: -0.1429)
მეზობელო კარისაო, სინათლე ხარ თვალისაო (Score: -0.4286)
ისეთი გოდორი დასწანი, რომ შენ შვილისშვილებსაც გამოადგესო (Score: -0.5000)
ვეშაპი ისეთს არაფერს ჩაყლაპავს, მონელება არ შეეძლოს. (Score: -0.5000)
კაცს როცა არ უნდა, რიყ